<a href="https://colab.research.google.com/github/harsh-sengar/Computer-Vision-Projects/blob/main/Vision_transformer_VOC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!nvidia-smi

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive',force_remount=True)

In [ ]:
cd /content/gdrive/MyDrive/Research/Object Detection/

In [ ]:
import xml.etree.ElementTree as et
import os
import cv2
import time
import copy
import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
import torchvision
from torchvision import datasets, models, transforms
from torch.utils.data import Dataset,DataLoader
import numpy as np
import matplotlib.pyplot as plt

from PIL import Image
from torch.utils.data.sampler import SubsetRandomSampler

from __future__ import print_function

import glob
from itertools import chain
import os
import random
import zipfile

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from PIL import Image
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from PIL import Image
from sklearn.model_selection import train_test_split
from torch.optim.lr_scheduler import StepLR
from torch.utils.data import DataLoader, Dataset
from torchvision import datasets, transforms
from tqdm.notebook import tqdm
from ipywidgets import IntProgress

import json
from PIL import Image
import PIL

import torch
from torchvision import transforms

In [ ]:
%%javascript

IPython.tab_as_tab_everywhere = function(use_tabs) {
    if (use_tabs === undefined) {
        use_tabs = true; 
    }

    // apply setting to all current CodeMirror instances
    IPython.notebook.get_cells().map(
        function(c) {  return c.code_mirror.options.indentWithTabs=use_tabs;  }
    );
    // make sure new CodeMirror instances created in the future also use this setting
    CodeMirror.defaults.indentWithTabs=use_tabs;

    };

IPython.tab_as_tab_everywhere()

In [ ]:
class VOC2007_dataset(Dataset):
  def __init__(self,label_dict,train_files,data_transforms=None):
    self.labels = [] # Labels of the object in the image
    self.paths = [] # Path of the image

    for i in train_files:
      tree = et.parse(i)
      path = './VOCdevkit/VOC2007/JPEGImages/'+i[i.rfind('/')+1:].split('.')[0]+'.jpg'
      self.paths.append(path)
      root = tree.getroot()
      self.label = [0]*21
      
      for child in root:
        if child.tag=='object':
          name = ''
          for child1 in child:
            if child1.tag=='name':
              name = child1.text
              self.label[label_dict[name]] = 1
      self.label = torch.as_tensor(self.label)
      self.labels.append(self.label) 

    self.n_samples = len(train_files)


  def __getitem__(self,index):
    img = Image.open(self.paths[index])
    if data_transforms:
      img = data_transforms(img)
    return img,self.labels[index]

  def __len__(self):
    return self.n_samples

In [ ]:
class VOC2007_dataset_test(Dataset):
  def __init__(self,label_dict,data_transforms=None):
    self.labels = []
    self.paths = []

    files = []

    for xyz in os.walk('./test/VOCdevkit/VOC2007/Annotations/'):
      path = xyz[0]
      for val in xyz[2]:
        files.append(path+val)
        

    for i in files:
      tree = et.parse(i)
      path = './test/VOCdevkit/VOC2007/JPEGImages/'+i[i.rfind('/')+1:].split('.')[0]+'.jpg'
      self.paths.append(path)
      root = tree.getroot()
      self.label = [0]*21

      for child in root:
        if child.tag=='object':
          name = ''
          for child1 in child:
            if child1.tag=='name':
              name = child1.text
              self.label[label_dict[name]] = 1
      self.label = torch.as_tensor(self.label)  
      self.labels.append(self.label)
    
    self.n_samples = len(files)


  def __getitem__(self,index):
    img = Image.open(self.paths[index])
    if data_transforms:
      img = data_transforms(img)

    return img,self.labels[index]

  def __len__(self):
    return self.n_samples

In [ ]:
f = open('./VOCdevkit/VOC2007/ImageSets/Main/train.txt')
train_files = []
for i in f.readlines():
    i = i.split('\n')[0]
    train_files.append('./VOCdevkit/VOC2007/Annotations/'+i+'.xml')

f1 = open('./VOCdevkit/VOC2007/ImageSets/Main/val.txt')
val_files = []
for i in f1.readlines():
    i = i.split('\n')[0]
    val_files.append('./VOCdevkit/VOC2007/Annotations/'+i+'.xml')

In [ ]:
train_files_temp = train_files
val_files_temp = val_files

In [ ]:
len(val_files_temp)

In [ ]:
len(train_files_temp)

In [ ]:
data_transforms = transforms.Compose([transforms.Resize((224,224)),
                                      transforms.ToTensor(),
                                      transforms.Normalize(mean=[0.5, 0.5, 0.5],
                                                          std = [0.5, 0.5, 0.5])])

In [ ]:
labels = {
 'aeroplane': 18,
 'bicycle': 1,
 'bird': 15,
 'boat': 6,
 'bottle': 16,
 'bus': 14,
 'car': 12,
 'cat': 11,
 'chair': 13,
 'cow': 19,
 'diningtable': 20,
 'dog': 5,
 'horse': 2,
 'motorbike': 10,
 'person': 4,
 'pottedplant': 7,
 'sheep': 8,
 'sofa': 9,
 'train': 3,
 'tvmonitor': 17}
dataset_train = VOC2007_dataset(label_dict=labels,train_files = train_files_temp,data_transforms=data_transforms)
dataset_val = VOC2007_dataset(label_dict=labels,train_files = val_files_temp,data_transforms = data_transforms)


In [ ]:
dataset_test = VOC2007_dataset_test(label_dict=labels,data_transforms=data_transforms)

In [ ]:
first_data = dataset_train[1]
print(first_data[1].shape)
print(first_data[1])
temp = first_data[0].numpy()
inp = temp.transpose((1, 2, 0))
mean = np.array([0.485, 0.456, 0.406])
std = np.array([0.229, 0.224, 0.225])
inp = std * inp + mean
inp = np.clip(inp, 0, 1)
plt.imshow(inp)

In [ ]:
first_data = dataset_val[1]
print(first_data[0].shape)
print(first_data[1])
temp = first_data[0].numpy()
inp = temp.transpose((1, 2, 0))
mean = np.array([0.485, 0.456, 0.406])
std = np.array([0.229, 0.224, 0.225])
inp = std * inp + mean
inp = np.clip(inp, 0, 1)
plt.imshow(inp)

In [ ]:
batch_size=32
train_loader = DataLoader(dataset_train, batch_size=batch_size, 
                                           shuffle=True,num_workers=8)
validation_loader = DataLoader(dataset_val, batch_size=batch_size,
                                                shuffle=True,num_workers=8)

test_loader = DataLoader(dataset_test,batch_size=batch_size,num_workers=8)
dataloaders = {}
dataloaders['train'] = train_loader
dataloaders['val'] = validation_loader
dataloaders['test'] = test_loader
dataset_sizes = {'train':len(dataset_train),'val':len(dataset_val),'test':len(dataset_test)}

In [ ]:
print(dataset_sizes)

In [ ]:
import os
import torch

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [ ]:
device

In [ ]:
def train_model_fine_tune(model, dataloaders, criterion, optimizer, labels_dict, num_epochs=25):
    since = time.time()

    val_acc_history = []

    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0

    for epoch in range(num_epochs):
        print('Epoch {}/{}'.format(epoch, num_epochs - 1))
        print('-' * 10)


        # Each epoch has a training and validation phase
        for phase in ['train', 'val']:
            if phase == 'train':
                model.train()  # Set model to training mode
            else:
                model.eval()   # Set model to evaluate mode

            running_loss = 0.0
            running_corrects = 0
            multi_label_acc = 0
            pre_label = [0]*21
            cat_label = [0]*21
            all_label = [0]*21
            # Iterate over data.
            for inputs, labels in dataloaders[phase]:
                inputs = inputs.to(device)
                labels = labels.to(device)

                # zero the parameter gradients
                optimizer.zero_grad()

                # forward
                # track history if only in train
                with torch.set_grad_enabled(phase == 'train'):
                    # Get model outputs and calculate loss
                    outputs = model(inputs)
                    labels = labels.type_as(outputs)
                    loss = criterion(outputs, labels)

                    # backward + optimize only if in training phase
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                # statistics
                running_loss += loss.item() * inputs.size(0)
                outputs = torch.sigmoid(outputs)
                predicted = torch.round(outputs)
                running_corrects += torch.sum(predicted == labels.data)

                predict_and_label = [p*l for p,l in zip(predicted,labels)]
                pre_or_label = [l+p for l,p in zip(predicted,labels)] 
                for pre in pre_or_label:
                    pre[pre!=0] = 1
                pre_label = [p+c for p,c in zip(pre_label, predict_and_label)]
                cat_label = [c+l for c,l in zip(cat_label,labels)]
                all_label = [a+l for a,l in zip(all_label,pre_or_label)]
                
                for a,o in zip(predict_and_label, pre_or_label):
                    if o.sum().item() != 0:
                        multi_label_acc = multi_label_acc + (a.sum().item()/o.sum().item())

            epoch_loss = running_loss / len(dataloaders[phase].dataset)
            
            pre_label = [ sum(x) for x in zip(*pre_label) ]
            cat_label = [ sum(x) for x in zip(*cat_label) ]
            all_label = [ sum(x) for x in zip(*all_label) ]
            pre_correct = 0
            cat_correct = 0
            all_pre_label = 0 
            

            acc_labels = []
            for p,c in zip(pre_label, cat_label):
                pre_correct = pre_correct + p.item()
                cat_correct = cat_correct + c.item()
                if c.item() != 0:
                    acc_labels.append(p.item()/c.item())
                else:
                    acc_labels.append(0)

            for a in all_label:
                all_pre_label = all_pre_label + a.item()

            label_accuracy = {}
            label_accuracy =  {k:v for k,v in zip(labels_dict.keys(),acc_labels[1:])}
            epoch_acc = (pre_correct/all_pre_label)*100
            print("Recall", (pre_correct/cat_correct)*100)
            print("Multi Label Accuracy", (multi_label_acc/len(dataloaders[phase].dataset))*100)
            print('{} Loss: {:.4f} Precision: {:.4f}'.format(phase, epoch_loss, epoch_acc))

            # deep copy the model
            if phase == 'val' and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model_wts = copy.deepcopy(model.state_dict())
            if phase == 'val':
                val_acc_history.append(epoch_acc)

        print()

    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(time_elapsed // 60, time_elapsed % 60))
    print('Best val Acc: {:4f}'.format(best_acc))

    # load best model weights
    model.load_state_dict(best_model_wts)
    return model, val_acc_history

In [ ]:
import timm
model_timm = timm.create_model('vit_base_patch16_224', pretrained=True, num_classes = 21).cuda()

In [ ]:
optimizer_ft = optim.Adam(model_timm.parameters(), lr=0.0001)
criterion_multi_label = nn.BCEWithLogitsLoss()

In [ ]:
model_ft, hist = train_model_fine_tune(model_timm, dataloaders, criterion_multi_label, optimizer_ft, labels, num_epochs=30)

In [ ]:
model_name = timm.list_models(pretrained=True)


In [ ]:
model_name

In [ ]:
plt.plot(range(80),train_loss,color='blue',label='training')
plt.plot(range(80),val_loss,color='green',label = 'validation')
plt.ylabel('loss')
plt.xlabel('Epochs')
plt.legend()

plt.title('lr=0.01, momentum=0.9) lr_scheduler.StepLR(optimizer_ft, step_size=50, gamma=0.05)')
plt.savefig('loss_epoch_curve.png')
plt.show()

train_acc1 = [float(i) for i in train_acc]
val_acc1 = [float(i) for i in val_acc]

plt.plot(range(80),train_acc1,color='blue',label='training')
plt.plot(range(80),val_acc1,color='green',label = 'validation')
plt.ylabel('Accuracy')
plt.xlabel('Epochs')
plt.legend()

plt.title('lr=0.01, momentum=0.9) lr_scheduler.StepLR(optimizer_ft, step_size=50, gamma=0.05)')
plt.savefig("accuracy_epoch_curve.png")
plt.show()

In [ ]:
phase = 'test'
running_loss = 0.0
since = time.time()
running_corrects = 0
for inputs, labels in dataloaders['test']:
  inputs = inputs.to(device)
  labels = labels.to(device)


                # forward
                # track history if only in train
  with torch.set_grad_enabled(phase == 'train'):
    outputs = model_ft(inputs)
    prob = nn.functional.softmax(outputs)
                    
    _, preds = torch.max(outputs, 1)
                    
    loss = criterion(outputs, labels)
                    
    if phase == 'train':
      loss.backward()
      optimizer.step()

                # statistics
  running_loss += loss.item() * inputs.size(0)
                
  running_corrects += torch.sum(preds == labels.data)
                

epoch_loss = running_loss / dataset_sizes[phase]
epoch_acc = running_corrects.double() / dataset_sizes['test']

print('{} Loss: {:.4f} Acc: {:.4f}'.format(
                phase, epoch_loss, epoch_acc))
            
time_elapsed = time.time() - since
print('Epoch complete in {:.0f}m {:.0f}s'.format(

time_elapsed // 60, time_elapsed % 60))

In [ ]:
def visualize_model(model, num_images=6):
    was_training = model.training
    model.eval()
    images_so_far = 0
    fig = plt.figure()

    with torch.no_grad():
        for i, (inputs, labels) in enumerate(dataloaders['test']):
            inputs = inputs.to(device)
            labels = labels.to(device)

            outputs = model(inputs)
            prob = nn.functional.softmax(outputs)

            print(prob.shape)
          
            prob,indices = torch.sort(prob,descending=True)
            prob = prob[:,:3]
            print(prob)
            _, preds = torch.max(outputs, 1)

            print(inputs.size())
            preds = preds.to("cpu").numpy()
            labels = labels.to("cpu").numpy()

            for j in range(inputs.size()[0]):
                images_so_far += 1
                ax = plt.subplot(num_images//2, 2, images_so_far)
                ax.axis('off')
                
                
                ax.set_title('predicted: {} {} {}'.format(class_names[preds[j]],prob[j],
                                                        class_names[labels[j]]))
                
                imshow(inputs[j].to("cpu"))

                if images_so_far == num_images:
                    model.train(mode=was_training)
                    return
        model.train(mode=was_training)

In [ ]:
visualize_model(model_ft)

In [ ]:
data_transforms = torchvision.transforms.Compose([torchvision.transforms.Resize((224,224))
                    ,transforms.ToTensor()]
)

In [ ]:
def intersection_union(b,b1):
  max_val=0
  x1_min = b[0]
  y1_min = b[1]
  x1_max = b[2]
  y1_max = b[3]
  
  x2_min = b1[:,0]
  y2_min = b1[:,1]
  x2_max = b1[:,2]
  y2_max = b1[:,3]

  #print(x1_min,y1_min,x1_max,y1_max,x2_min,y2_min,x2_max,y2_max)
  

  x1 = np.minimum(x1_max,x2_max)
  x2 = np.maximum(x1_min,x2_min)
  y1 = np.minimum(y1_max,y2_max)
  y2 = np.maximum(y1_min,y2_min)

  #print(x1,x2,y1,y2)

  x_dist = np.maximum(0,(x1 - x2))
  y_dist = np.maximum(0,(y1-y2))

  intersect = x_dist * y_dist
  
  union = (x1_max-x1_min)*(y1_max-y1_min)+(x2_max-x2_min)*(y2_max-y2_min)-intersect

  return intersect/union

def calc_mAP(predictions,dataset1,iou_thresh,num_classes):

  average_precisions = []
  for class_c in range(1,num_classes):
    filter_predictions = []
    filter_true_box = {}
    total_ground_truth=0
  
    for i in predictions:
      f = i[0]
      pred_class = i[1]
      #pred_boxes = i[3]
    
      for j,lab in enumerate(pred_class):
        if labels_dict[lab]==class_c:          
          filter_predictions.append([f,class_c,i[2][j],i[3][j]])
        
      true_class = dataset1.data1[f][1]
      true_boxes = dataset1.data1[f][0]
      filter_true_box[f] = []

      for j in range(len(true_class)):
        if true_class[j]==class_c:
          filter_true_box[f].append(true_boxes[j])
          total_ground_truth+=1
      
      
    TP = torch.zeros(len(filter_predictions))
    FP = torch.zeros(len(filter_predictions))
    #taken = torch.zeros(len(filter_true_box))
    taken = {}
    for key in filter_true_box:
      taken[key] = torch.zeros(len(filter_true_box[key]))
    
    

    #print('filter_true_box ',filter_true_box,'\ntotal ground truth ',total_ground_truth)
    filter_predictions.sort(key=lambda x:x[2],reverse=True)
    
    for j,prediction in enumerate(filter_predictions):

      f = prediction[0]
      true_boxes = np.array(filter_true_box[f]) 
      #print('true_boxes ',type(true_boxes),true_boxes)
      
      if true_boxes.shape[0]==0:
        FP[j]=1
        continue

      iou_overlap = intersection_union(prediction[3],true_boxes)

      #print(iou_overlap)
      max_iou = np.max(iou_overlap,0)
      idx = np.argmax(iou_overlap)

      if max_iou >= iou_thresh:
        
        if taken[f][idx]==0:
          TP[j]=1
          taken[f][idx]=1
      
        else:
          FP[j] = 1

      else:
        FP[j] = 1
      
    TP_cumsum = torch.cumsum(TP,dim=0)
    FP_cumsum = torch.cumsum(FP,dim=0)
    recalls = TP_cumsum/total_ground_truth
    precisions = TP_cumsum/(TP_cumsum+FP_cumsum)
    precisions = torch.cat((torch.tensor([1]), precisions))
    recalls = torch.cat((torch.tensor([0]), recalls))
    average_precisions.append(torch.trapz(precisions,recalls))
    #print(recalls,'\n', precisions)
    plt.plot(recalls,precisions)
    #plt.xticks(np.arange(0,1.1,0.1))
    #plt.yticks(np.arange(0,1.1,0.1))
    plt.xlabel('recalls')
    plt.ylabel('precisions')
    plt.title(class_names[class_c])
    plt.savefig('{}.png'.format(class_names[class_c]))
    plt.show()

    

  print(sum(average_precisions)/len(average_precisions))


calc_mAP(filter_detections,dataset_val,0.3,21)





  

    




In [ ]:
Image.open('./VOCdevkit/VOC2007/JPEGImages/000005.jpg')

In [ ]:
img = Image.open('./VOCdevkit/VOC2007/JPEGImages/006524.jpg')
data_transforms = torchvision.transforms.Compose([torchvision.transforms.Resize((256)),
                                                  transforms.RandomVerticalFlip(),
                                                  transforms.RandomHorizontalFlip(),
                                                  transforms.RandomAffine(degrees=0,translate=(0.25,0.25))])

#temp = data_transforms(img)
w,h = img.size
diff=0
diff1=0
w1=0
w2=0
h1=0
h2=0
a=0
b=0
while diff<64:
  a = np.random.random()
  a1 = int(a*w)
  b = np.random.random()
  b1 = int(b*w)
  w1 = np.minimum(a1,b1)
  w2 = np.maximum(a1,b1)
  diff = w2-w1
#print(w1,w2,a,b)
while diff1<64:
  a = np.random.random()
  a1 = int(a*h)
  b = np.random.random()
  b1 = int(b*h)
  h1 = np.minimum(a1,b1)
  h2 = np.maximum(a1,b1)
  diff1 = h2-h1
#print(h1,h2,a,b)
img.crop((w1,h1,w2,h2))
